In [50]:
import numpy as np
import mindspore
import mindspore.nn as nn
import mindspore.ops as ops
from mindspore import Parameter, Tensor, ms_function

In [51]:
def make_batch(sentences, word_dict):
    input_batch = []
    target_batch = []
    
    for sent in sentences:
        word = sent.split()
        inp = [word_dict[n] for n in word[:-1]]
        tgt = word_dict[word[-1]]
        
        input_batch.append(inp)
        target_batch.append(tgt)
    return input_batch, target_batch

In [52]:
class NNLM(nn.Cell):
    def __init__(self, n_steps, vocab_size, embed_size, hidden_size):
        super().__init__()
        self.C = nn.Embedding(vocab_size, embed_size)
        self.H = nn.Dense(n_steps * embed_size, hidden_size, has_bias=False)
        self.d = Parameter(Tensor(np.ones(hidden_size), mindspore.float32), name='d')
        self.U = nn.Dense(hidden_size, vocab_size, has_bias=False)
        self.W = nn.Dense(n_steps * embed_size, vocab_size, has_bias=False)
        self.b = Parameter(Tensor(np.ones(vocab_size), mindspore.float32), name='b')
        self.n_steps = n_steps
        self.embed_size = embed_size
        self.tanh = nn.Tanh()

    def construct(self, X):
        X = self.C(X)
        X = X.view(-1, self.n_steps * self.embed_size)
        tanh = self.tanh(self.d + self.H(X))
        output = self.b + self.W(X) + self.U(tanh)
        return output
        

In [53]:
n_steps = 2
hidden_size = 2
embed_size = 2

sentences = ["i like dog", "i love coffee", "i hate milk"]

word_list = " ".join(sentences).split()
word_list = list(set(word_list))
word_dict = {w: i for i, w in enumerate(word_list)}
number_dict = {i: w for i, w in enumerate(word_list)}
vocab_size = len(word_dict)

In [54]:
input_batch, target_batch = make_batch(sentences, word_dict)
input_batch = Tensor(input_batch, mindspore.int32)
target_batch = Tensor(target_batch, mindspore.int32)
target_batch

Tensor(shape=[3], dtype=Int32, value= [1, 6, 3])

In [55]:
model = NNLM(n_steps, vocab_size, embed_size, hidden_size)

In [56]:
criterion = nn.CrossEntropyLoss()
optimizer = nn.Adam(model.trainable_params(), learning_rate=0.001)

In [57]:
def forward(inputs, targets):
    logits = model(inputs)
    loss = criterion(logits, targets)
    return loss

In [58]:
grad_fn = ops.value_and_grad(forward, None, optimizer.parameters)

In [59]:
@ms_function
def train_step(inputs, targets):
    loss, grads = grad_fn(inputs, targets)
    optimizer(grads)
    return loss

In [60]:
model.set_train()

epoch = 5000
for step in range(epoch):
    loss = train_step(input_batch, target_batch)
    if (step + 1) % 1000 == 0:
        print('Epoch:', '%04d' % (step + 1), 'cost = ', '{:.6f}'.format(loss.asnumpy()))

Epoch: 1000 cost =  0.159208
Epoch: 2000 cost =  0.016804
Epoch: 3000 cost =  0.005246
Epoch: 4000 cost =  0.002221
Epoch: 5000 cost =  0.001076


In [61]:
model.set_train(False)
predict = model(input_batch).asnumpy().argmax(axis=1)
print(predict)
print([sen.split()[:2] for sen in sentences], '->', [number_dict[n.item()] for n in predict])

[1 6 3]
[['i', 'like'], ['i', 'love'], ['i', 'hate']] -> ['dog', 'coffee', 'milk']
